In [180]:
import polars as pl
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import gc
import time

In [181]:
# Step 1: Read Data with Lags
def read_data_with_lags(current_file_path, previous_file_path, columns_needed, lag_column="responder_6", lag_name="responder_6_lag_1", length = 5000):
    """Read the current file and add lagged data from the previous file."""
    # Read current data
    current_file = pl.scan_parquet(current_file_path).select(columns_needed).head(length).collect()
    
    # If there is a previous file, add lagged data
    if previous_file_path:
        previous_file = pl.scan_parquet(previous_file_path).select(["time_id", lag_column]).head(length).collect()
        lagged_data = previous_file.rename({lag_column: lag_name})
        current_file = current_file.join(lagged_data, on="time_id", how="left")
    
    return current_file

# Step 2: Clean Data
def clean_data(df):
    """Clean the data by handling missing values."""
    if isinstance(df, pl.DataFrame):
        df = df.fill_null(strategy="backward").fill_null(strategy="forward").fill_null(0)
    else:
        raise TypeError("The input is not a Polars DataFrame.")
    return df

# Step 3: Preprocess Data
def preprocess_data(df, target_column='responder_6', n_components=0.95):
    """Apply PCA to reduce dimensions."""
    # Drop the target column and convert to numpy
    X = df.drop(target_column).to_numpy()
    y = df[target_column].to_numpy()

    # Apply PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, y

# Step 4: Anonymize
def anonymize(file): 
    return file

In [182]:
from xgboost import XGBRegressor

def select_features_by_importance(df, target_column='responder_6', threshold=0.01):
 
    X = df.drop(target_column).to_pandas()
    y = df[target_column].to_pandas()
    
    # Train a temporary XGBoost model to get feature importance
    model = XGBRegressor(objective='reg:squarederror', n_estimators=50, random_state=42)
    model.fit(X, y)
    
    # Get feature importance
    feature_importances = model.feature_importances_
    important_features = X.columns[feature_importances >= threshold]
    
    # Filter the original DataFrame for these features
    filtered_df = df.select(list(important_features) + [target_column])
    return filtered_df


Model Functions

In [183]:
# Step 5: Train Model
from sklearn.model_selection import cross_val_score

def train_XGB(X, y):
    """Train the model using XGBoost."""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
    mean_mae = -np.mean(cv_scores)
    print(f"Cross-Validated Mean Absolute Error (MAE): {mean_mae}")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error (MAE): {mae}")
    return model

In [184]:
def train_SVM():
    pass

In [ ]:
def pipeline(current_file_path, previous_file_path, columns_needed ,model, counter):

    start_time = time.time()
    df = read_data_with_lags(current_file_path, previous_file_path, columns_needed)
    df = clean_data(df)
    df = anonymize(df)
    df = select_features_by_importance(df, target_column='responder_6', threshold=0.001)

    X, y = preprocess_data(df, target_column='responder_6', n_components=0.95)
    
    if model =='XGB':
        model = train_XGB(X, y)
    elif model == 'SVM': 
        mode = train_SVM(X,y)
    else: 
        raise NameError('Model name not found')

    # Clear memory
    del df, X, y
    gc.collect()  # Force garbage collection
    
    end_time = time.time()
    print(f"Finished processing file {counter+1}/{len(file_paths)} in {end_time - start_time:.2f} seconds\n")
    
    # Update previous file path
    previous_file_path = current_file_path

    return previous_file_path, model


In [186]:
columns_needed = [
    'date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
    'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
    'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
    'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
    'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
    'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
    'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
    'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
    'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
    'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
    'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
    'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
    'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61',
    'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66',
    'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71',
    'feature_72', 'feature_73', 'feature_74', 'feature_75', 'feature_76',
    'feature_77', 'feature_78', 'responder_6'
]

Call Functions 

In [187]:

# Jupyter Notebook - File Paths 
file_paths = [
    f"/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet" for i in range(10)]

# Train XGB with PCA via pipeline without storing all in memory
previous_file_path = None
for i, current_file_path in enumerate(file_paths):
    previous_file_path, model = pipeline(current_file_path, previous_file_path, columns_needed, 'XGB', i)

# Train SVM with PCA via pipeline without storing all in memory
'''previous_file_path = None
for i, current_file_path in enumerate(file_paths):
    previous_file_p''ath, model = pipeline(current_file_path, previous_file_path, columns_needed, 'SVM', i)'''

Cross-Validated Mean Absolute Error (MAE): 0.5403464436531067
Mean Absolute Error (MAE): 0.18809212744235992
Finished processing file 1/10 in 1.47 seconds

Cross-Validated Mean Absolute Error (MAE): 0.760183310508728
Mean Absolute Error (MAE): 0.15672925114631653
Finished processing file 2/10 in 1.62 seconds

Cross-Validated Mean Absolute Error (MAE): 0.8766644120216369
Mean Absolute Error (MAE): 0.1739712655544281
Finished processing file 3/10 in 1.95 seconds

Cross-Validated Mean Absolute Error (MAE): 0.9394318580627441
Mean Absolute Error (MAE): 0.1807750016450882
Finished processing file 4/10 in 1.97 seconds

Cross-Validated Mean Absolute Error (MAE): 1.0900036454200746
Mean Absolute Error (MAE): 0.230383962392807
Finished processing file 5/10 in 2.01 seconds

Cross-Validated Mean Absolute Error (MAE): 0.5448825597763062
Mean Absolute Error (MAE): 0.11643332988023758
Finished processing file 6/10 in 2.02 seconds

Cross-Validated Mean Absolute Error (MAE): 0.8487887978553772
Mean Ab

"previous_file_path = None\nfor i, current_file_path in enumerate(file_paths):\n    previous_file_p''ath, model = pipeline(current_file_path, previous_file_path, columns_needed, 'SVM', i)"

In [188]:
# git add .
# git commit -m "commit message"
# git push origin main